In [1]:
from stock_selector import DataFetcher, DataCleaner, DataHandler, StockSelector
import numpy as np
import pandas as pd
import sys
import os

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


In [2]:
conn = DataFetcher().conn

In [3]:
query = f"""
SELECT * FROM JYDB.LC_InstiArchive
"""
df = pd.read_sql(query, conn)

In [2]:
dataFetcher = DataFetcher()
dataHandler = DataHandler()
data_file_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
base_pool = pd.read_csv(data_file_path + '//codes_map.csv')
calender = dataFetcher.get_calender_data('2014-01-01', '2023-08-01', endlevel=[(1,2), (1), (1,2), (1,2)])

In [3]:
mv_data = pd.read_csv(data_file_path + '//daily_avg_mv_data.csv')
mv_data = mv_data.set_index('wind_code')
turnoverV_data = pd.read_csv(data_file_path + '//daily_avg_turnoverV_data.csv')
turnoverV_data = turnoverV_data.set_index('wind_code')
mv_rank = DataHandler().top_rank(mv_data, 0.8)
turnoverV_rank = DataHandler().top_rank(turnoverV_data, 0.8)
dividend_rank = pd.read_csv(data_file_path + '//dividend_index_data.csv')
dividend_rank = dividend_rank.set_index('wind_code')
dividend_rank = dividend_rank.apply(lambda col: col[col==1].index).apply(pd.Series).T
stock_pool = []
for i in range(115):
    common_stocks = np.intersect1d(mv_rank.iloc[:,i].dropna(), np.intersect1d(turnoverV_rank.iloc[:,i].dropna(), dividend_rank.iloc[:,i].dropna()))
    stock_pool.append(common_stocks)
final_result = pd.DataFrame(stock_pool).T

In [4]:
payratio_data = DataFetcher().get_payratio_data(calender)
payratio_data = DataHandler().payratio_handler(payratio_data, base_pool)

In [5]:
dividend_data = DataFetcher().get_dividend_data(start_year=2011, end_year=2024)

In [6]:
dividend_data = DataHandler().dividend_delta_handler(dividend_data,base_pool)

In [7]:
dividend_data = dividend_data.reset_index()

In [8]:
final_result.columns = payratio_data.columns

In [9]:
test = StockSelector(DataFetcher().conn).filter_1(final_result, payratio_data, dividend_data)